In [2]:
#Montaje de Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Lectura y preprocesamiento de imágenes

In [3]:

#Importación de librerías y configuración
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Directorio de datasets
dataset_path = '/content/drive/My Drive/proyecto2024_datasets/1.chest-xray-pneumonia/chest_xray/train'

# Tamaño al que se redimencionan las imágenes para que sean uniformes
IMG_SIZE = (64, 64)

# Función para cargar imágenes y convertirlas en arrays
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path).convert('L')  # Convertir a escala de grises
            img = img.resize(IMG_SIZE)  # Redimensionar las imágenes
            img_array = np.array(img).flatten()  # Convertir la imagen a un vector 1D
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f'Error al procesar {filename}: {e}')
    return images, labels

# Cargar imágenes de las carpetas 'NORMAL' y 'PNEUMONIA' asignando etiquetas (0 para NORMAL y 1 para PNEUMONIA).
normal_images, normal_labels = load_images_from_folder(os.path.join(dataset_path, 'NORMAL'), 0)
pneumonia_images, pneumonia_labels = load_images_from_folder(os.path.join(dataset_path, 'PNEUMONIA'), 1)

# Combinar las imágenes y las etiquetas
X = normal_images + pneumonia_images
y = normal_labels + pneumonia_labels

# Convertir a arrays de NumPy
X = np.array(X)
y = np.array(y)

# Separar en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)





##Expl:

* `os`: Utilizado para manejar rutas de archivos y directorios.
* `numpy`: Usado para manejar arrays numéricos, esencial para la manipulación de datos.
* `PIL`: Biblioteca para abrir, manipular y guardar imágenes.
* `train_test_split`: Función de sklearn para dividir los datos en conjuntos de entrenamiento y validación.

`def load_images_from_folder(folder, label):`
* Objetivo: Cargar imágenes desde una carpeta, preprocesarlas y convertirlas en vectores.
* `Image.open(img_path).convert('L')`: Abre la imagen y la convierte a escala de grises (simplifica la imagen a un solo canal de color).
* `img.resize(IMG_SIZE)`: Redimensiona la imagen al tamaño especificado.
* `np.array(img).flatten()`: Convierte la imagen en un array NumPy y la aplana a un vector 1D. **Resultado**: Al convertir la imagen a un array de NumPy, se crea una representación de la imagen donde cada píxel tiene un valor asociado (generalmente entre 0 y 255 en el caso de imágenes en escala de grises). **Por ejemplo**, si la imagen tiene un tamaño de 64x64 pixeles, el array que se obtiene es de la forma (64, 64).
* `.flatten()`: convierte un array de más de una dimensión en un vector de una sola dimensión (1D). Los modelos de machine learning, como SVM, requieren que las entradas sean vectores de una sola dimensión. La imagen original es 2D (64x64), y al "aplanarla" con .flatten(), se convierte en un vector de 1D con 64 × 64 = 4096 elementos.

El vector de una dimensión será lo que el modelo utilizará como entrada para hacer las predicciones.


##Entrenamiento y evaluación

In [4]:


from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Modelo SVM
svm_model = SVC(kernel='linear') #Crea un modelo de SVM (Support Vector Machine) con un kernel lineal.
svm_model.fit(X_train, y_train) #Entrena el modelo con los datos de entrenamiento

# Predicciones en el conjunto de validación
y_pred = svm_model.predict(X_val)

# Evaluación del modelo
print("SVM Accuracy:", accuracy_score(y_val, y_pred)) #Calcula la precisión general del modelo (proporción de predicciones correctas)
print(classification_report(y_val, y_pred)) #informe detallado con presicion, recall, f1 score para ambas clases.


SVM Accuracy: 0.9330085261875761
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       862
           1       0.92      0.94      0.93       780

    accuracy                           0.93      1642
   macro avg       0.93      0.93      0.93      1642
weighted avg       0.93      0.93      0.93      1642



## Expl.:

`svm_model = SVC(kernel='linear')` crea un modelo de **Máquinas de Soporte Vectorial (SVM)** en Scikit-Learn utilizando un **kernel lineal**.

`SVC`: Es la clase para crear un clasificador de SVM en Scikit-Learn. `SVC` significa "Support Vector Classifier" (Clasificador de Vectores de Soporte).

`kernel='linear'`: El parámetro `kernel` especifica el tipo de función que usará SVM para separar los datos. En este caso, se utiliza un kernel lineal, lo que significa que SVM intentará encontrar una línea recta (en 2D) que separe las clases en los datos.

Cuando se entrena el modelo con los datos, el algoritmo de SVM busca la mejor línea o hiperplano que maximice la margen entre las diferentes clases.


# Pruebas en el conjunto de test


In [5]:
# Ruta de la carpeta de test
test_dir = '/content/drive/My Drive/proyecto2024_datasets/1.chest-xray-pneumonia/chest_xray/test'

# Cargar las imágenes de la carpeta de test
test_normal_images, test_normal_labels = load_images_from_folder(os.path.join(test_dir, 'NORMAL'), 0)
test_pneumonia_images, test_pneumonia_labels = load_images_from_folder(os.path.join(test_dir, 'PNEUMONIA'), 1)

# Combinar las imágenes y etiquetas de test
X_test = test_normal_images + test_pneumonia_images
y_test = test_normal_labels + test_pneumonia_labels

# Convertir a arrays de NumPy
X_test = np.array(X_test)
y_test = np.array(y_test)

# Realizar predicciones sobre el conjunto de test
y_test_pred = svm_model.predict(X_test)

# Evaluar el rendimiento del modelo sobre el conjunto de test
print("SVM Accuracy (Test):", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


SVM Accuracy (Test): 0.8546562228024369
              precision    recall  f1-score   support

           0       0.98      0.80      0.88       759
           1       0.71      0.96      0.82       390

    accuracy                           0.85      1149
   macro avg       0.84      0.88      0.85      1149
weighted avg       0.89      0.85      0.86      1149



# Pruebas manuales

In [25]:
image_path = '/content/drive/My Drive/proyecto2024_datasets/1.chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0033-0001.jpeg'

def preprocess_image(image_path):
    IMG_SIZE = (64, 64)
    try:
        img = Image.open(image_path).convert('L')
        img = img.resize(IMG_SIZE)
        img_array = np.array(img).flatten()
        return np.array([img_array])
    except Exception as e:
        print(f"Error al procesar la imagen: {e}")
        return None

image_to_predict = preprocess_image(image_path)


In [26]:
if image_to_predict is not None:
    prediction = svm_model.predict(image_to_predict)
    label = "NORMAL" if prediction == 0 else "PNEUMONIA"
    print(f"La imagen seleccionada ha sido clasificada como: {label}")
else:
    print("No se pudo procesar la imagen.")


La imagen seleccionada ha sido clasificada como: NORMAL
